In [31]:
#Run through all the cases in a specified folder
#Look through all the times, generate NIFTI files using reconstruction
#Put it into a separate output folder
#Each case must include the ED and ES in a info.txt

In [32]:
import os
import numpy as np
from PIL import Image
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import map_coordinates 
from glob import glob

In [33]:
realigned_folder = "C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/Finial_Cases(37&#x3a;52)(image+mask)"
output_folder = "C:/Users/haziq/OneDrive - Imperial College London/Modules/Fourth Year/MEng Individual Project/MastersProject/GHD/Motion_Niftis"

output_results = os.path.join(output_folder, "results")
output_image = os.path.join(output_folder,"image")

In [34]:
print([os.path.basename(case) for case in glob(realigned_folder+"/*") if os.path.isdir(case)])

['07SEP2021P1_1', '07SEP2021P1_3', '07SEP2021P2_1', '07SEP2021P3_1', '12OCT2021P1_2', '12OCT2021P2_3', '12OCT2021P2_4', '13JUL2021P1_1', '13JUL2021P1_3', '21092021P2_1', '21092021P2_2', '21092021P2_3', '21225_2', '23NOV2021P3_1', 'c.h._5', 'dtga_vitium_1', 'dtga_vitium_2', 'dtga_vitium_3', 'I0000068', 'IMG_20241006_2_1', 'IMG_20241006_2_2', 'IMG_20241006_2_3', 'IMG_20241006_2_4', 'IMG_20241006_2_5', 'IMG_20241206_2_1', 'IMG_20241206_2_2', 'IMG_20241212_2_2', 'IMG_20241212_2_3', 'IMG_20241230_4_2', 'IMG_20241230_4_3', 'IMG_20250326_6_1', 'IMG_20250326_6_2', 'IMG_20250328_3_1', 'IMG_20250328_3_2', 'k2025_12', 'k2025_13', 'k2025_5', 'k2025_7', 'l.k._4', 'stic2002_5', 'stic2002_6', 'stic2025-1_1', 'stic2025-1_2', 'stic2025-1_3', 'stic2025-1_4', 'stic2025-1_5', 'stic2025-1_6', 'stic21225_2', 'stic21225_3', 'stic21225_5', 'TOF_05AUG2016P1A', 'TOF_24JUN2016P1']


In [35]:
cases = [os.path.basename(case) for case in glob(realigned_folder+"/*") if os.path.isdir(case)]
done_cases = [os.path.basename(case) for case in glob(os.path.join(output_folder, "Done/*")) if os.path.isdir(case)]
for case in cases[:]:  # iterate over a copy
    if case in done_cases:
        cases.remove(case)
        continue
    
    os.makedirs(os.path.join(output_results,case),exist_ok=True)
    os.makedirs(os.path.join(output_image,case),exist_ok=True)

print(cases)
print(done_cases)


['21092021P2_3', '21225_2', 'c.h._5', 'dtga_vitium_1', 'dtga_vitium_2', 'dtga_vitium_3', 'I0000068', 'IMG_20241006_2_3', 'IMG_20241006_2_4', 'IMG_20241006_2_5', 'IMG_20241206_2_1', 'IMG_20241206_2_2', 'IMG_20241212_2_2', 'IMG_20241212_2_3', 'IMG_20241230_4_2', 'IMG_20241230_4_3', 'IMG_20250326_6_1', 'IMG_20250326_6_2', 'IMG_20250328_3_1', 'IMG_20250328_3_2', 'k2025_12', 'k2025_13', 'k2025_5', 'k2025_7', 'l.k._4', 'stic2002_5', 'stic2002_6', 'stic2025-1_1', 'stic2025-1_2', 'stic2025-1_3', 'stic2025-1_4', 'stic2025-1_5', 'stic2025-1_6', 'stic21225_2', 'stic21225_3', 'stic21225_5', 'TOF_05AUG2016P1A', 'TOF_24JUN2016P1']
['07SEP2021P1_1', '07SEP2021P1_3', '07SEP2021P2_1', '07SEP2021P3_1', '12OCT2021P1_2', '12OCT2021P2_3', '12OCT2021P2_4', '13JUL2021P1_1', '13JUL2021P1_3', '21092021P2_1', '21092021P2_2', '23NOV2021P3_1', 'IMG_20241006_2_1', 'IMG_20241006_2_2']


In [36]:
print(len(cases))

38


In [37]:
def load_images_from_time(input_folder, type, downsample_factor=1):
    file_list = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])
    slices = []
    
    for file in file_list:
        img = cv2.imread(os.path.join(input_folder, file), 0)
        img = img / img.max()
        
        if type != "img":
            vals = np.unique(img)
            if len(vals) > 2:  # Ensure there are at least 3 unique values
                img[img == vals[1]] = 255
                img[img == vals[2]] = 127
        
        if downsample_factor > 1:
            new_size = (img.shape[1] // downsample_factor, img.shape[0] // downsample_factor)
            img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

        slices.append(np.array(img))
    
    return slices[:37]  # Ensure only the first 36 images are loaded

    # image = cv2.imread(glob(user_path)[0],0)
# print(image.shape)

In [38]:
def cylindrical_volume(slices, angle_step, centroid=None, use_interpolation=True):
    # Assume each slice is a 2D numpy array with shape (height, width)
    height, width = slices[0].shape
    # print(f"Slice dimensions: {(height, width)}")
    
    #Default centroid to image center
    center_x = center_y = centroid
    
    # Use a square base dimension that is the maximum of width and height.
    base = width
    # Create the volume with dimensions (base, base, height).
    # The x-y plane is square to fully capture rotated coordinates.
    volume = np.zeros((base, base, height), dtype=slices[0].dtype)
    
    # Process each slice.
    for i, slice_img in enumerate(slices):
        theta = np.deg2rad(i * angle_step)
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)
        
        # Loop over pixel positions relative to the image center.
        for dx in range(-center_x, center_x):
            for dy in range(-center_y, center_y):
                # Compute the rotated coordinates for the horizontal offset (dx).
                # These determine where the pixel falls in the x-y plane of the volume.
                x_rot = int(base // 2 + dx * cos_theta)
                y_rot = int(base // 2 + dx * sin_theta)
                # Use the vertical offset (dy) to determine the z coordinate.
                z_pos = dy + center_y  # This will be in [0, height)
                
                # Ensure the computed indices are within the volume bounds.
                if 0 <= x_rot < base and 0 <= y_rot < base and 0 <= z_pos < height:
                    # Map the pixel from the slice into the volume.
                    volume[x_rot, y_rot, z_pos] = slice_img[dy + center_y, dx + center_x]
    
    return volume

In [39]:
# Step 3: Save as NIfTI
def save_nifti(volume, output_file):
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))
    nib.save(nifti_img, output_file)
    print(f"NIfTI file saved to {output_file}")

In [40]:
def create_nifti(volume_path, output_path, type="img"):
    slices = np.array(load_images_from_time(volume_path, type,1))
    centroid = np.shape(slices[0])[1]//2
    _, n_r, n_z = np.shape(slices)
    n_theta = 360
    volume = np.zeros((n_theta, n_r, n_z))
    
    for theta_idx,slice in zip(range(0, 180, 5), slices):  # 180 degrees, 5 degree steps
        # Compute the slice index (i corresponds to theta in degrees)

        volume[theta_idx, :, :] = slice
        
    cylindrical_vol = cylindrical_volume(slices,5,centroid, False)
    save_nifti(cylindrical_vol, output_path)

In [41]:
print(cases)

['21092021P2_3', '21225_2', 'c.h._5', 'dtga_vitium_1', 'dtga_vitium_2', 'dtga_vitium_3', 'I0000068', 'IMG_20241006_2_3', 'IMG_20241006_2_4', 'IMG_20241006_2_5', 'IMG_20241206_2_1', 'IMG_20241206_2_2', 'IMG_20241212_2_2', 'IMG_20241212_2_3', 'IMG_20241230_4_2', 'IMG_20241230_4_3', 'IMG_20250326_6_1', 'IMG_20250326_6_2', 'IMG_20250328_3_1', 'IMG_20250328_3_2', 'k2025_12', 'k2025_13', 'k2025_5', 'k2025_7', 'l.k._4', 'stic2002_5', 'stic2002_6', 'stic2025-1_1', 'stic2025-1_2', 'stic2025-1_3', 'stic2025-1_4', 'stic2025-1_5', 'stic2025-1_6', 'stic21225_2', 'stic21225_3', 'stic21225_5', 'TOF_05AUG2016P1A', 'TOF_24JUN2016P1']


In [ ]:
for case in cases:
    # print(case)
    # # Images
    # case_image = os.path.join(realigned_folder,"images", case)
    # image_times = [folder for folder in glob(case_image+"/*") if "time" in folder]
    # image_output = os.path.join(output_folder, f"images/{case}")
    # # print(image_output)
    # for time in image_times:
    #     output = os.path.join(image_output, f"{os.path.basename(case)}_{os.path.basename(time)}.nii.gz")
    #     print(time)
    #     print(output)
    #     if os.path.isfile(output):
    #         continue
    #     break
    # break
    #     # create_nifti(time,output)       
            
    #Results
    # case_results = os.path.join(realigned_folder,"results", case)
    case_results = os.path.join(realigned_folder, case)
    results_times = [folder for folder in glob(case_results+"/*") if "time" in folder]
    results_output = os.path.join(output_folder, f"results/{case}")
    # Assign the results times to ED and ES
    for time in results_times:

        output = os.path.join(results_output, f"{os.path.basename(case)}_{os.path.basename(time)}.nii.gz")
        #Segmentations
        segmentation_path = os.path.join(time,"mask")
        seg_output = os.path.join(results_output, f"{os.path.basename(case)}_{os.path.basename(time)}_seg.nii.gz")
        # print(segmentation_path, seg_output)
        if  os.path.isfile(seg_output):
            continue

        # if os.path.isfile(seg_output):
        #     continue
        # create_nifti(time,output)   
        # print(segmentation_path)
        # print(seg_output)
        print(segmentation_path)
        create_nifti(segmentation_path, seg_output)

C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/Finial_Cases(37&#x3a;52)(image+mask)\dtga_vitium_2\time001\mask
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Modules/Fourth Year/MEng Individual Project/MastersProject/GHD/Motion_Niftis\results/dtga_vitium_2\dtga_vitium_2_time001_seg.nii.gz
C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/Finial_Cases(37&#x3a;52)(image+mask)\dtga_vitium_2\time002\mask
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Modules/Fourth Year/MEng Individual Project/MastersProject/GHD/Motion_Niftis\results/dtga_vitium_2\dtga_vitium_2_time002_seg.nii.gz
C:/Users/haziq/OneDrive - Imperial College London/Directional Data/QIFENG/Finial_Cases(37&#x3a;52)(image+mask)\dtga_vitium_2\time003\mask
NIfTI file saved to C:/Users/haziq/OneDrive - Imperial College London/Modules/Fourth Year/MEng Individual Project/MastersProject/GHD/Motion_Niftis\results/dtga_vitium_2\dtga_vitium_2